In [2]:
import cv2
from sklearn import svm
import numpy as np
import glob
from skimage.feature import greycomatrix, greycoprops
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt
import functools


ModuleNotFoundError: No module named 'cv2'

In [36]:
def get_all_glcm_features(gray_scale_img):
    
    #size of co-occ matrix = number of gray levels
    image_array = np.array(gray_scale_img)
    #print('first pixel= ', image_array[0][0])
    coocurrence_matrix = greycomatrix(image_array, [1], [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4])
    contrast = greycoprops(coocurrence_matrix, 'contrast')
    homogeneity = greycoprops(coocurrence_matrix, 'homogeneity')
    #mean = greycoprops(coocurrence_matrix, 'mean')
    energy = greycoprops(coocurrence_matrix, 'energy')
    #entropy = greycoprops(coocurrence_matrix, 'entropy')
    #variance = greycoprops(coocurrence_matrix, 'variance')
    correlation = greycoprops(coocurrence_matrix, 'correlation')
    return contrast, homogeneity, energy, correlation

In [37]:
def Preprocessing (image) : 
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    height, width = image.shape
    img = cv2.GaussianBlur(image, (9, 9), 0) #decrease noise for dialation
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 101, 30)
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 101, 30)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 20))
    img = cv2.dilate(img, kernel, iterations=8)
    contours = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0] 
    biggest_contour = functools.reduce(lambda c1, c2: c1 if cv2.contourArea(c1) > cv2.contourArea(c2) else c2,contours) #find the biggest contour for text area
    x, y, w, h = cv2.boundingRect(biggest_contour) # find smallest rect that can contain the text area after dialation
    image = image[y:y + h, x:x + w]
    return image


[0.01730614 0.0246604  0.01289823 0.02066758 0.99134693 0.9876698
 0.99355089 0.98966621 0.94120776 0.93742761 0.94346913 0.93945855
 0.82180902 0.74623261 0.86721223 0.78732066]


In [39]:
image = cv2.imread("..\\Data_split\\train\\Females\\F1.jpg")
img = Preprocessing(image)
contrast, homogeneity, energy, correlation= get_all_glcm_features((img * 255).astype(np.uint8))
features=[]
features.append(contrast.ravel())
features.append(homogeneity.ravel())
features.append(energy.ravel())
features.append(correlation.ravel())
features=(np.array(features)).ravel()
print(features)

[0.01045092 0.01509454 0.00916938 0.01373563 0.99477454 0.99245273
 0.99541531 0.99313218 0.96143524 0.95905161 0.96207619 0.95973941
 0.83995662 0.76903166 0.85963255 0.78982498]
